In [2]:
import pandas as pd
import numpy as np
import mitosheet
import json

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.9) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
input_data = "../5b-Geocoding/Canada Fire Depts Rec Type 2-3 12.csv"
df_input = pd.read_csv(input_data, low_memory=False, dtype="str")

In [125]:
# df4 = df_input[df_input['latitude'].isna()]
# print('number to geocode:', len(df4))

# df5 = df4[~df4.street_no.isna()]
# print('unable to geocode: ', len(df5))

number to geocode: 345150
unable to geocode:  241060


In [8]:
# provinces = df_input['province'].unique()

FDID = []
ST_NO = []
ST_NAME = []
ST_NAME_2 = []
CITY = []
PROV = []
POST = []
LATS = []
LONS = []

df = pd.DataFrame(columns=['FDID','gc_street_no','gc_street_name', \
                               'gc_city','gc_province','gc_postal_code', \
                               'gc_latitude', 'gc_longitude'])

# for province_code in provinces:
#     print(province_code)
json_file = 'fire_departments.json'

try:
    with open(json_file, 'r', encoding='utf-8') as f:
        JSONS=json.load(f)  
#     print(province_code, '- rows: ', len(JSONS))

    # add json data to arrays
    for index, element in enumerate(JSONS):
        if element==[]:
            append_blank(index)

        else:
            # format gc api results      
            if 'meta' in element:
                if (element['data'][0]['country']['code'] == 'CA'):
                    FDID.append(element['idx'])
                    if not element['data'][0]['civicNumber']['number'] is None:
                        ST_NO.append(element['data'][0]['civicNumber']['number'])
                    else:
                        ST_NO.append('')
                        print('oops')
                    ST_NAME.append(element['data'][0]['streetName'])
                    ST_NAME_2.append(element['data'][1]['streetName'])
                    CITY.append(element['data'][0]['cityName']['en'])
                    PROV.append(element['data'][0]['mailingAddress']['province']['code'])
                    POST.append(element['data'][0]['mailingAddress']['postalCode'])
                    LATS.append(element['data'][0]['location']['geoCoordinates']['latitude'])
                    LONS.append(element['data'][0]['location']['geoCoordinates']['longitude'])
    #                 if 'streetName' in element.keys():
    #                     ST_NAME.append(element['streetName'])
    #                 else:
    #                     ST_NAME.append('nope')
            else:
                FDID.append('')
                LATS.append('')
                LONS.append('')
                ST_NO.append('nope')
                ST_NAME.append('')
                CITY.append('')
                PROV.append('')
                POST.append('')
                ST_NAME_2.append('')
except FileNotFoundError:
    print('no file: ', json_file)


# append results to dataframe
df['FDID'] = FDID
df['gc_street_no'] = ST_NO
df['gc_street_name'] = ST_NAME
df['gc_street_name_2'] = ST_NAME_2
df['gc_city'] = CITY
df['gc_province'] = PROV
df['gc_postal_code'] = POST
df['gc_latitude'] = LATS
df['gc_longitude'] = LONS

In [9]:
df_json = df

In [13]:
df2 = pd.merge(df_input, df_json, how='left', on='FDID')
len(df2)

3697

In [36]:
mitosheet.sheet(df2, analysis_to_replay="id-gvaywzbnal")

MitoWidget(analysis_data_json='{"analysisName": "id-zqspjwhksi", "analysisToReplay": {"analysisName": "id-gvay…

In [ ]:
from mitosheet import *; register_analysis("id-gvaywzbnal");
    
# Reordered column street_address
df2_columns = [col for col in df2.columns if col != 'street_address']
df2_columns.insert(6, 'street_address')
df2 = df2[df2_columns]

# Reordered column city_town
df2_columns = [col for col in df2.columns if col != 'city_town']
df2_columns.insert(8, 'city_town')
df2 = df2[df2_columns]

# Reordered column zip_code
df2_columns = [col for col in df2.columns if col != 'zip_code']
df2_columns.insert(10, 'zip_code')
df2 = df2[df2_columns]

# Reordered column state
df2_columns = [col for col in df2.columns if col != 'state']
df2_columns.insert(7, 'state')
df2 = df2[df2_columns]

# Reordered column gc_street_no
df2_columns = [col for col in df2.columns if col != 'gc_street_no']
df2_columns.insert(4, 'gc_street_no')
df2 = df2[df2_columns]

# Reordered column gc_street_no
df2_columns = [col for col in df2.columns if col != 'gc_street_no']
df2_columns.insert(3, 'gc_street_no')
df2 = df2[df2_columns]

# Reordered column city_town
df2_columns = [col for col in df2.columns if col != 'city_town']
df2_columns.insert(5, 'city_town')
df2 = df2[df2_columns]

# Sorted keep_gc in ascending order
df2 = df2.sort_values(by='keep_gc', ascending=True, na_position='first')

# Sorted keep_gc in descending order
df2 = df2.sort_values(by='keep_gc', ascending=False, na_position='last')


In [42]:
# Only keep GC addresses if they match
df2 = df2.fillna('')
def find_gc(row):
    num_string = str(row.gc_street_no)
    return (num_string in row.street_address and
            row.gc_street_name.lower() in row.street_address.lower())

df2['keep_gc'] = df2.apply(find_gc, axis=1)
# df2['address'] = df2['address'].where(df2['keep_gc'], '')

In [63]:
# Data summaries

total = len(df2)
keeps = len(df2[df2['keep_gc'] == True])
percent_keep = 100 * keeps / total
print('percent of good results: ', np.round(percent_keep, 0))

# Count the number of rows in the 'address' column that contain the string 'box'
box_count = df2.loc[df2['street_address'].str.contains('box', case=False), 'street_address'].count()
percent_box = 100 * box_count / total
print('percent of good results: ', np.round(percent_box, 0))

box_count = df2.loc[df2['street_address'].str.contains('box', case=False), 'street_address'].count()

# Create a boolean mask indicating whether the street_address column is empty or does not contain a number
mask = df2['street_address'].isnull() | ~df2['street_address'].str.contains('\d')

# Use the boolean mask to select the rows that match the condition
selected_rows = df2[mask]

# Count the number of rows
num_rows = len(selected_rows)

print(f'Number of rows: {np.round(100 * num_rows/ total, 0)}')

percent of good results:  21.0
percent of good results:  43.0
Number of rows: 9.0


In [131]:
# For testing

# df2 = df2[['idx', 'street_no', 'gc_street_no', 'street_name', 
#           'gc_street_name', 'gc_street_name_2', 'latitude', 'gc_latitude', 
#           'longitude', 'gc_longitude']]
# df3 = df2.sample(300)


In [64]:
df2.to_csv('fire_departments_geocoded.csv', index=False)

In [ ]:
# Have to do the loop to add in duplicates addresses

In [35]:
# We only keep GC API results if the street number 
# and name matches the query

# if the street numbers match
# and the gc_street_name is contained in street_name
# for i in range(len(df2)):
#     num_string = str(df2.at[i,'gc_street_no'])
#     if (num_string in df2.at[i,'street_address'] and
#        df2.at[i,'gc_street_name'].lower() in df2.at[i,'street_address'].lower()):
#         df2.at[i,'keep_gc'] = 'yes'










In [40]:
total = len(df2)
keeps = len(df2[df2['keep_gc'] == True])
percent_matches = 100 * keeps / total
print('percent of good results: ', np.round(percent_matches, 0))

percent of good results:  21.0


In [138]:
df = df2

In [143]:
# Now we want to add in GC lat lons for any matches that were 
# removed by deduplication


df.sort_values(['province', 'city', 'street_no', 'formatted_en'], inplace=True)
# df.reset_index(inplace=True)

for i in range(len(df)):
    
    # check if the street number is the same as the previous row
    if (i > 1):
        if (df.at[i , 'street_no'] == df.at[i - 1, 'street_no']):
            # then street name
            if (df.at[i, 'formatted_en'] == df.at[i - 1, 'formatted_en']):
                # then city
                if (df.at[i, 'city'] == df.at[i - 1, 'city']):
                    # then province
                    if (df.at[i, 'province'] == df.at[i - 1, 'province']):
                        
                        # if they are all the same and we have an gc latitude,
                        # then assign it to the previous row
                        
                        # check if previous row had a gc latitude
                        gc_longitude = df.at[i - 1, 'gc_latitude']
                        if (not pd.isna(gc_longitude) and gc_longitude != 0):
                            # if yes, then assign it to the current row
                            df.at[i, 'gc_latitude'] = gc_longitude
                            
                            # do the same for y
                            gc_longitude = df.at[i - 1, 'gc_longitude']
                            if (not pd.isna(gc_longitude) and gc_longitude != 0):
                                df.at[i, 'gc_longitude'] = gc_longitude
                                
                            # do the same for gc keep match 
                            keep_gc = df.at[i - 1, 'keep_gc']
                            if (not pd.isna(keep_gc) and keep_gc != 0):
                                df.at[i, 'keep_gc'] = keep_gc
                    
                            # also add info other columns from the matching stage
#                             df.at[i, 'keep_gc'] = df.at[i - 1, 'keep_gc']



In [144]:
total = len(df)
keeps = len(df[df['keep_gc'] == 'yes'])
percent_matches = 100 * keeps / total
print('percent of good results: ', np.round(percent_matches, 0))

percent of good results:  20.0


In [149]:
# df['geo_source'] = np.where(isinstance(df[gc_longitude], float), "gc_match", df['geo_source'])
df['geo_source'] = np.where(df['keep_gc'] == 'yes', "gc_match", df['geo_source'])
df['latitude'] = np.where(df['keep_gc'] == 'yes', df['gc_latitude'], df['latitude'])
df['longitude'] = np.where(df['keep_gc'] == 'yes', df['gc_longitude'], df['longitude'])

# df['longitude'] = np.where(not df[gc_longitude].isna(), df['gc_longitude'], df['longitude'])

In [155]:
# Finally, we clean up columns that are not being used
df = df.drop(columns=['gc_street_no', 'gc_street_name', 'gc_street_name_2', \
'gc_city', 'gc_province', 'gc_postal_code', 'gc_latitude', 'gc_longitude'])

df = df.drop(columns=['x', 'y', 'ratio', 'matches_r'])



In [156]:
df.to_csv('data/output/geocoded.csv', index=False)

In [154]:
mitosheet.sheet(df.sample(1000), analysis_to_replay="id-wpvpckaimu")

MitoWidget(analysis_data_json='{"analysisName": "id-imwgfljpks", "analysisToReplay": {"analysisName": "id-wpvp…

In [158]:
df['geo_source'].value_counts()

oda_match    235747
Source       223612
gc_match     159985
Name: geo_source, dtype: int64

In [159]:
print(len(df[df['latitude'].isna()]))

185165


In [160]:
print(len(df[df['geo_source'].isna()]))

184147


In [161]:
print(len(df[df['street_no'].isna()]))

106548
